# Results comparison between Trajectron++ and our method
Comparing the results of our method with the Trajectron++ paper's method. The goal of the comparison is to evaluate Trajectron++ on a different dataset than it was trained on to allow more fair comparison.

In [60]:
import sys
import os
import dill
import json
import argparse
import pandas as pd
import numpy as np
import generative_model
import matplotlib.pyplot as plt
from tqdm import tqdm

## Helper functions

In [82]:
def calculate_FDE(pred_x, pred_y, test_x, test_y):

    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    return FDE

def calculate_ADE(pred_x, pred_y, test_x, test_y):
    total_displacement_error = 0
    for point_idx in range(len(test_x)):
        displacement_error = np.sqrt((pred_x[point_idx] - test_x[point_idx])**2 + (pred_y[point_idx] - test_y[point_idx])**2)
        total_displacement_error += displacement_error

    return total_displacement_error/len(pred_x)

## The evaluation logic for Trajectron++ loops over the frames and predicts the future trajectories 
## for each node present in the current frame
## Each node has to have at least 7 historical points and 12 future points
def get_total_predictable_slices(data):
    total_predictable_steps = 0
    for i in pd.unique(data.node_id):
        #print(len(test[test.node_id == i]))
        total_predictable_steps += len(data[data.node_id == i]) - 19
    return total_predictable_steps

In [4]:
def process_data(input_data):
    data = input_data.copy()
    data['frame_id'] = pd.to_numeric(data['frame_id'], downcast='integer')
    data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')

    data['frame_id'] = data['frame_id'] // 10

    data['frame_id'] -= data['frame_id'].min()

    data['node_type'] = 'PEDESTRIAN'
    data['node_id'] = data['track_id'].astype(str)
    data.sort_values('frame_id', inplace=True)

    data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
    data['pos_y'] = data['pos_y'] - data['pos_y'].mean()
    
    # Select only such nodes which have enough data to predict on (7 historical timesteps, 12 future)
    v = data.node_id.value_counts()
    data = data[data.node_id.isin(v.index[v.gt(19)])]
    
    return data

## Method evaluation logic

In [136]:
def evaluate_our_method(data, params, dataset_title='', single_output = False):
    tot = 0
    our_fde_best_of = []
    our_ade_best_of = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='Ours - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y, _ = generative_model.predict(x_data, y_data, params, trajectory_length=12)

                    best_fde = None
                    best_ade = None
                    
                    if single_output:
                        avg_x = np.mean(all_pred_x, axis=0)
                        avg_y = np.mean(all_pred_y, axis=0)
                        best_fde = calculate_FDE(avg_x, avg_y, x_gt, y_gt)
                        best_ade = calculate_ADE(avg_x, avg_y, x_gt, y_gt)
                    else:
                        for i in range(len(all_pred_x)):
                            current_pred_x = all_pred_x[i]
                            current_pred_y = all_pred_y[i]

                            fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                            if best_fde == None or fde < best_fde:
                                best_fde = fde

                            ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                            if best_ade == None or ade < best_ade:
                                best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of

In [137]:
def evaluate_cvm_with_scenarios(data, dataset_title='', history_length=8):
    tot = 0
    our_fde_best_of = []
    our_ade_best_of = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='CVM - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y = [], []
                    
                    # CVM
                    for i in range(20):
                        history_x = x_data[-history_length:]
                        history_y = y_data[-history_length:]
                        assert len(history_x) == history_length
                        
                        if i == 0:
                            vel_x = [history_x[i] - history_x[i-1] for i in range(1, len(history_x))]
                            vel_y = [history_y[i] - history_y[i-1] for i in range(1, len(history_y))]
                        else:
                            vel_x = [(history_x[i] - history_x[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_x))]
                            vel_y = [(history_y[i] - history_y[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_y))]
                        
                        assert len(vel_x) == history_length-1
                        avg_vel_x = np.mean(vel_x)
                        avg_vel_y = np.mean(vel_y)
                        
                        pred_x = [x_data[-1] + i*avg_vel_x for i in range(1, 13)]
                        pred_y = [y_data[-1] + i*avg_vel_y for i in range(1, 13)]
                        assert len(pred_x) == 12
                        
                        all_pred_x.append(pred_x)
                        all_pred_y.append(pred_y)
                    

                    best_fde = None
                    best_ade = None
                    for i in range(len(all_pred_x)):
                        current_pred_x = all_pred_x[i]
                        current_pred_y = all_pred_y[i]

                        fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_fde == None or fde < best_fde:
                            best_fde = fde

                        ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_ade == None or ade < best_ade:
                            best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of

## Automated results comparison

In [140]:
params_eth = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.8,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.5,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.2,
    "VELOCITY_CHANGE_NOISE": 0.1,

    "ANGLE_CHANGE_PROB": 0.2,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.15, 0.68, 0.95, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 7, 7, 5] # Total 20 traj
}

params_rest = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.9,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.5,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.15,
    "VELOCITY_CHANGE_NOISE": 0.1,

    "ANGLE_CHANGE_PROB": 0.1,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.25, 0.68, 0.95, 1.0],
    "GROUP_CLUSTER_COUNT": [2, 7, 6, 5] # Total 20 traj
}

In [141]:
our_method_params = [params_eth, params_eth, params_rest, params_rest]

files = [
    'eth/test/biwi_eth.txt', 
    'hotel/test/biwi_hotel.txt', 
    'zara1/test/crowds_zara01.txt', 
    'zara2/test/crowds_zara02.txt'
]

trajectron_resultset_names = [
    'eth_vel_12', 
    'hotel_vel', 
    'zara1_vel', 
    'zara2_vel'
]

In [143]:
def evaluate_all_datasets(our_method_params, files, trajectron_resultset_names):
    base_path = './pedestrians/raw/'

    ours_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_long_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_short_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}

    for file_idx, file in enumerate(files):
        data = pd.read_csv(base_path + file, sep='\t', index_col=False, header=None)
        data.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

        data = process_data(data)

        num_predictable_trajectories = get_total_predictable_slices(data)

        ## Trajectron
        trajectron_fde = []
        with open('pedestrians/results/' + trajectron_resultset_names[file_idx] + '_fde_best_of.csv', mode='r') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            line_count = 0
            for row in csv_reader:
                trajectron_fde.append(float(row['value']))

        trajectron_ade = []
        with open('pedestrians/results/' + trajectron_resultset_names[file_idx] + '_ade_best_of.csv', mode='r') as csv_file:
            csv_reader = csv.DictReader(csv_file)
            line_count = 0
            for row in csv_reader:
                trajectron_ade.append(float(row['value']))

        trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))

        # make sure that there is no discrepancy between our data processing and trajectron evaluation results size
        assert len(trajectron_fde) == num_predictable_trajectories
        assert len(trajectron_ade) == num_predictable_trajectories

        ## Ours
        our_fde_best_of_20, our_ade_best_of_20 = evaluate_our_method(data, our_method_params[file_idx], dataset_title=trajectron_resultset_names[file_idx])
        ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_best_of_20))
        ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_best_of_20))

        ## CVM
        cvm_fde_best_of, cvm_ade_best_of = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[file_idx])
        cvm_fde_short_history_best_of, cvm_ade_short_history_best_of = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[file_idx], history_length=2)
        cvm_long_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_best_of))
        cvm_long_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_best_of))
        cvm_short_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_short_history_best_of))
        cvm_short_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_short_history_best_of))
    
    
    ## Univ is a separate case as it has 2 scenes
    filename1 = './pedestrians/raw/univ/test/students001.txt'
    filename2 = './pedestrians/raw/univ/test/students003.txt'
    univ_data_1 = pd.read_csv(filename1, sep='\t', index_col=False, header=None)
    univ_data_2 = pd.read_csv(filename2, sep='\t', index_col=False, header=None)
    univ_data_1.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']
    univ_data_2.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

    univ_data_1 = process_data(univ_data_1)
    univ_data_2 = process_data(univ_data_2)

    ## Ours
    our_fde_best_of_1, our_ade_best_of_1 = evaluate_our_method(univ_data_1, params_rest, dataset_title='univ 1')
    our_fde_best_of_2, our_ade_best_of_2 = evaluate_our_method(univ_data_2, params_rest, dataset_title='univ 2')

    our_fde_best_of_20 = our_fde_best_of_1 + our_fde_best_of_2
    our_ade_best_of_20 = our_ade_best_of_1 + our_ade_best_of_2

    ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_best_of_20))
    ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_best_of_20))

    ## CVM
    cvm_fde_best_of_1, cvm_ade_best_of_1 = evaluate_cvm_with_scenarios(univ_data_1, dataset_title='univ 1')
    cvm_fde_best_of_2, cvm_ade_best_of_2 = evaluate_cvm_with_scenarios(univ_data_2, dataset_title='univ 2')

    cvm_fde_best_of = cvm_fde_best_of_1 + cvm_fde_best_of_2
    cvm_ade_best_of = cvm_ade_best_of_1 + cvm_ade_best_of_2

    cvm_fde_short_hist_best_of_1, cvm_ade_short_hist_best_of_1 = evaluate_cvm_with_scenarios(univ_data_1, dataset_title='univ 1', history_length=2)
    cvm_fde_short_hist_best_of_2, cvm_ade_short_hist_best_of_2 = evaluate_cvm_with_scenarios(univ_data_2, dataset_title='univ 2', history_length=2)

    cvm_fde_short_history_best_of = cvm_fde_short_hist_best_of_1 + cvm_fde_short_hist_best_of_2
    cvm_ade_short_history_best_of = cvm_ade_short_hist_best_of_1 + cvm_ade_short_hist_best_of_2

    cvm_long_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_best_of))
    cvm_long_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_best_of))
    cvm_short_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_short_history_best_of))
    cvm_short_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_short_history_best_of))

    ## Trajectron
    trajectron_fde = []
    with open('pedestrians/results/univ_vel_fde_best_of.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_fde.append(float(row['value']))

    trajectron_ade = []
    with open('pedestrians/results/univ_vel_ade_best_of.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_ade.append(float(row['value']))

    trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
    trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
    
    return [
        ours_results,
        trajectron_results,
        cvm_long_results,
        cvm_short_results
    ]

In [ ]:
res = evaluate_all_datasets(our_method_params, files, trajectron_resultset_names)

ours_results = res[0]
trajectron_results = res[1]
cvm_long_results = res[2]
cvm_short_results = res[3]

Ours - univ 1:  12%|███████████▎                                                                                   | 53/444 [05:21<59:25,  9.12s/it]

In [131]:
index = [
    'ETH', 
    'Hotel', 
    'Zara 1', 
    'Zara 2',
    'Univ'
]

df_data_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['FDE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['FDE'], index = index)
}

df_fde = pd.DataFrame(df_data_fde)

df_data_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['ADE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['ADE'], index = index)
}

df_ade = pd.DataFrame(df_data_ade)

In [132]:
#df_fde
df_fde.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Trajectron++,Ours
ETH,1.119766,1.768382,0.918525,0.892368
Hotel,0.366464,0.571324,0.273470,0.264365
Zara 1,0.796770,0.881716,0.399671,0.509057
Zara 2,0.555702,0.654918,0.313920,0.413604
Univ,0.887809,1.058825,0.545510,0.582854


In [133]:
df_ade.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Trajectron++,Ours
ETH,0.654488,0.897325,0.534569,0.562136
Hotel,0.205606,0.300796,0.173194,0.162743
Zara 1,0.415939,0.406821,0.210878,0.304891
Zara 2,0.294794,0.303181,0.162733,0.247927
Univ,0.475241,0.492425,0.278476,0.353319


### Testing with an additional group

In [ ]:
params_eth = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.8,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.5,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.2,
    "VELOCITY_CHANGE_NOISE": 0.1,

    "ANGLE_CHANGE_PROB": 0.2,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.15, 0.40, 0.68, 0.95, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 6, 5, 5, 3] # Total 20 traj
}

params_rest = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.9,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.5,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.15,
    "VELOCITY_CHANGE_NOISE": 0.1,

    "ANGLE_CHANGE_PROB": 0.1,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.15, 0.40, 0.68, 0.95, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 6, 5, 5, 3] # Total 20 traj
}

res = evaluate_all_datasets(our_method_params, files, trajectron_resultset_names)

ours_results = res[0]
trajectron_results = res[1]
cvm_long_results = res[2]
cvm_short_results = res[3]

In [ ]:
index = [
    'ETH', 
    'Hotel', 
    'Zara 1', 
    'Zara 2',
    'Univ'
]

df_data_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['FDE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['FDE'], index = index)
}

df_fde = pd.DataFrame(df_data_fde)

df_data_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['ADE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['ADE'], index = index)
}

df_ade = pd.DataFrame(df_data_ade)